# Assign Sheet's variables

In [ ]:
SPREADSHEET_ID = "1ZhMBwrAzyMlc2L9RXfAvXmXdEvR5BDeHX_H1-bgpIUw"
subSheetName = "businesses"

# Create Indices file

In [ ]:
!touch indices.json

# Process Data

In [ ]:
import json
import pandas as pd

input_file = "./indices.json"
with open(input_file, 'r') as file:
    data = json.load(file)

In [ ]:
rows = []
for index_name, keys in data.items():
    row = [index_name]
    for key, value in keys.items():
        row.append(f"{key}{'-' if value == -1 else ''}")  # Prefix '-' if value is -1
    rows.append(row)

max_columns = max(len(row) for row in rows)

rows = [row + [''] * (max_columns - len(row)) for row in rows]

In [ ]:
columns = ['Index Name'] + [f"Key{i+1}" for i in range(max_columns - 1)]
indices = pd.DataFrame(rows, columns=columns)

In [ ]:
indices.sort_values(
    by=columns[1:],
    key=lambda col: col.str.rstrip('-'),
    ignore_index=True,
    inplace=True
)


In [ ]:
singleIndexCount = len(indices[indices['Key2'] == ""])
compoundIndexCount = len(indices[indices['Key2'] != ""])

In [ ]:
indices['singleIndexCount'] = [singleIndexCount] + [""] * (len(indices) - 1)
indices['compoundIndexCount'] = [compoundIndexCount] + [""] * (len(indices) - 1)

# Export Data

In [ ]:
# output_path = "./indices.csv"
# indices.to_csv(output_path, index=False)

# Update spreadSheet

In [ ]:
from googleapiclient.discovery import build
from google.auth import default
from google.colab import auth
auth.authenticate_user()


creds, _ = default()
service = build('sheets', 'v4', credentials=creds)

data = indices.values.tolist()
data.insert(0, indices.columns.tolist())


body = {
    'values': data
}
service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=subSheetName,
    valueInputOption="USER_ENTERED",
    body=body
).execute()

print("Google Sheet updated successfully!")

Google Sheet updated successfully!


# Highlight index names with similar Key1

In [ ]:
split_indices = indices['Key1'].str.rsplit('-', n=1).str[0]

duplicatedIndices = split_indices[split_indices.duplicated()].unique()

In [ ]:
import colorsys

def generate_distinct_colors(num_colors):
    colors = []
    for i in range(num_colors):
        hue = i / num_colors
        lightness = 0.6
        saturation = 0.8
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append('{:02X}{:02X}{:02X}'.format(int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)))
    return colors


In [ ]:
from googleapiclient.discovery import build

spreadsheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
sheets = spreadsheet.get('sheets', [])
sheet_id = None

for sheet in sheets:
    if sheet['properties']['title'] == subSheetName:
        sheet_id = sheet['properties']['sheetId']
        break

if sheet_id is None:
    raise ValueError(f"Sheet '{subSheetName}' not found in the spreadsheet.")


sheet_data = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=subSheetName).execute()
rows = sheet_data.get('values', [])


key1_colors = {}

# Define a distinct color palette
color_palette = generate_distinct_colors(len(duplicatedIndices))


key1_colors = {}
requests = []
start_row = 1


for row_idx, row in enumerate(rows[start_row:], start=start_row):
    if len(row) > 1:
      key1_value = row[1].rstrip('-').strip()
      if key1_value in duplicatedIndices:  # Ensure there's a value in the Key1 column
          if key1_value not in key1_colors:
              key1_colors[key1_value] = color_palette[len(key1_colors) % len(color_palette)]

          # Convert hex color to RGB
          color_hex = key1_colors[key1_value]
          red = int(color_hex[0:2], 16) / 255
          green = int(color_hex[2:4], 16) / 255
          blue = int(color_hex[4:6], 16) / 255

          # Create request for updating background color
          requests.append({
              'updateCells': {
                  'rows': [{
                      'values': [{
                          'userEnteredFormat': {
                              'backgroundColor': {
                                  'red': red,
                                  'green': green,
                                  'blue': blue,
                              }
                          }
                      }]
                  }],
                  'fields': 'userEnteredFormat.backgroundColor',
                  'range': {
                      'sheetId': sheet_id,
                      'startRowIndex': row_idx,
                      'endRowIndex': row_idx+1,
                      'startColumnIndex': 0,
                      'endColumnIndex': 1,  # Adjust as necessary
                  }
              }
          })
      else:
        requests.append({
                'updateCells': {
                    'rows': [{
                        'values': [{
                            'userEnteredFormat': {
                                'backgroundColor': {
                                    'red': 1,
                                    'green': 1,
                                    'blue': 1,
                                }
                            }
                        }]
                    }],
                    'fields': 'userEnteredFormat.backgroundColor',
                    'range': {
                        'sheetId': sheet_id,
                        'startRowIndex': row_idx,
                        'endRowIndex': row_idx+1,
                        'startColumnIndex': 0,
                        'endColumnIndex': 1,  # Adjust as necessary
                    }
                }
            })

body = {'requests': requests}


In [ ]:
%%capture
service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body=body).execute()

In [ ]:
sheet_url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/edit#gid={sheet_id}"
print(f"Sheet URL: {sheet_url}")

Sheet URL: https://docs.google.com/spreadsheets/d/1ZhMBwrAzyMlc2L9RXfAvXmXdEvR5BDeHX_H1-bgpIUw/edit#gid=1617756355
